In [159]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, cohen_kappa_score
import mne

In [160]:
# ==================== CONFIG ====================
DATA_DIR = 'BCICIV_2a/'          # Folder containing A01T.gdf ... A09E.gdf
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32
EPOCHS = 50
LR = 0.001
T_MIN = 0.0      # Start of epoch relative to cue
T_MAX = 4.0      # 4-second trials (standard for this dataset)

In [161]:
class EEGDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)   # [trials, channels, time]
        self.labels   = torch.tensor(labels,   dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [162]:
class BiGRUModel(nn.Module):
    def __init__(self, num_channels=22, num_classes=4, hidden_dim=128, dropout=0.5):
        super().__init__()
        # Spatial convolution to reduce channel dimension (common in EEG DL)
        self.spatial = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(num_channels, 1), stride=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout2d(dropout)
        )

        self.bigru = nn.GRU(input_size=32,
                            hidden_size=hidden_dim,
                            num_layers=2,
                            batch_first=True,
                            bidirectional=True,
                            dropout=dropout)

        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        # x: [batch, channels, time]
        batch = x.size(0)
        x = x.unsqueeze(1)                     # [batch, 1, channels, time]
        x = self.spatial(x)                    # [batch, 32, 1, time]
        x = x.squeeze(2)                       # [batch, 32, time]
        x = x.permute(0, 2, 1)                 # [batch, time, 32] for GRU
        _, hn = self.bigru(x)                  # hn: [4, batch, hidden] (2 layers × bidirectional)
        hn = hn.view(2, 2, batch, -1)          # separate layers & directions
        hn = hn[-1]                            # take top layer
        hn = hn.transpose(0, 1).contiguous().view(batch, -1)  # [batch, hidden*2]
        out = self.classifier(hn)
        return out

In [ ]:
def load_subject_data(subject_id, data_dir):
    def preprocess_raw(raw):
        raw.drop_channels(['EOG-left', 'EOG-central', 'EOG-right'])
        raw.filter(8., 30.)                     # Mu/beta band
        raw.set_eeg_reference('average')
        return raw

    sessions = ['T', 'E']
    X_list, y_list = [], []

    for sess in sessions:
        path = os.path.join(data_dir, f'{subject_id}{sess}.gdf')
        
        try:
            raw = mne.io.read_raw_gdf(path, preload=True, eog=['EOG-left', 'EOG-central', 'EOG-right'])
        except FileNotFoundError:
            print(f"  File not found: {path}")
            continue
            
        raw = preprocess_raw(raw)
        
        # Get events from annotations
        events, event_id_from_annotations = mne.events_from_annotations(raw)
        
        if sess == 'T':
            mi_codes = []
            for code in [769, 770, 771, 772, 1, 2, 3, 4]:
                if code in events[:, 2]:
                    mi_codes.append(code)
                    print(f"  Found {np.sum(events[:, 2] == code)} events with code {code}")
            
            if not mi_codes:
                print("  No standard MI codes found, checking all codes...")
                # Find codes that occur around the cue (783)
                cue_indices = np.where(events[:, 2] == 783)[0]
                for idx in cue_indices:
                    if idx + 1 < len(events):
                        next_code = events[idx + 1, 2]
                        if next_code not in mi_codes and next_code not in [768, 783]:
                            mi_codes.append(next_code)
            
            print(f"  Motor imagery codes identified: {mi_codes}")
            
            if not mi_codes:
                print(f"  ERROR: Could not identify motor imagery events!")
                continue
            
            # Create event_id mapping
            event_id = {}
            # Sort codes to assign labels consistently
            mi_codes_sorted = sorted(mi_codes)
            labels = ['left', 'right', 'foot', 'tongue']
            
            for i, code in enumerate(mi_codes_sorted[:4]):  # Take up to 4 classes
                event_id[labels[i]] = code
            
            print(f"  Using event_id mapping: {event_id}")
            
        else:  # sess == 'E'
            # ===== EVALUATION SESSION =====
            # In evaluation, we often don't have class labels
            # Or they might be in a different format
            
            print(f"\n  Evaluation session - looking for cues and labels...")
            
            # For BCI IV-2a evaluation, we need to use a different approach
            # Typically, we epoch based on cue onset (783) and use provided labels
            
            # First, let's see what we have
            cue_indices = np.where(events[:, 2] == 783)[0]
            print(f"  Found {len(cue_indices)} cue onset events (783)")
            
            if len(cue_indices) == 0:
                print("  No cue events found, cannot create epochs")
                continue
            
            # For evaluation, we need the true labels from a separate file
            # Usually there's a .mat or .txt file with the true labels
            label_file = os.path.join(data_dir, f'true_labels_{subject_id}_E.mat')
            
            if os.path.exists(label_file):
                # Load true labels from MATLAB file
                import scipy.io
                mat_data = scipy.io.loadmat(label_file)
                true_labels = mat_data['classlabel'].flatten() - 1  # Convert to 0-3
                print(f"  Loaded {len(true_labels)} true labels from file")
                
                # We'll create epochs using cue onsets and assign these labels
                # For now, create dummy event_id for cue
                event_id = {'cue': 783}
            else:
                print(f"  No true label file found at {label_file}")
                print(f"  Cannot process evaluation session without labels")
                continue
        
        # Create epochs for both sessions (approach differs)
        picks = mne.pick_types(raw.info, eeg=True, eog=False)
        
        if sess == 'T':
            # Create epochs based on motor imagery events
            epochs = mne.Epochs(raw, events, event_id=event_id,
                               tmin=0, tmax=4,
                               picks=picks, 
                               baseline=None,
                               preload=True,
                               reject_by_annotation=False)
            
            X = epochs.get_data()
            y = epochs.events[:, -1]
            
            # Map to consistent labels
            y_mapped = []
            code_to_label = {769: 0, 770: 1, 771: 2, 772: 3, 
                           1: 0, 2: 1, 3: 2, 4: 3}
            
            for code in y:
                if code in code_to_label:
                    y_mapped.append(code_to_label[code])
                else:
                    # Skip if not a valid MI code
                    continue
            
            # Filter to keep only valid epochs
            valid_indices = [i for i, code in enumerate(y) if code in code_to_label]
            X = X[valid_indices]
            y_mapped = np.array(y_mapped)
            
        else:  # E session
            # Create epochs based on cue onset (783)
            # We need to create custom events for cue onsets
            cue_events = events[events[:, 2] == 783]
            
            if len(cue_events) != len(true_labels):
                print(f"  Warning: {len(cue_events)} cues but {len(true_labels)} labels")
                # Take the minimum
                n_trials = min(len(cue_events), len(true_labels))
                cue_events = cue_events[:n_trials]
                true_labels = true_labels[:n_trials]
            
            # Create epochs using cue events
            epochs = mne.Epochs(raw, cue_events, event_id={'cue': 783},
                               tmin=0, tmax=4,
                               picks=picks, 
                               baseline=None,
                               preload=True,
                               reject_by_annotation=False)
            
            X = epochs.get_data()
            y_mapped = true_labels[:len(X)]  # Match length
            
            print(f"  Created {len(X)} evaluation epochs")
        
        if len(X) > 0:
            X_list.append(X)
            y_list.append(y_mapped)
            print(f"  Created {len(X)} epochs for session {sess}")
        else:
            print(f"  No valid epochs created for session {sess}")
    
    # Handle return
    if len(X_list) == 2:
        X_train, X_test = X_list[0], X_list[1]
        y_train, y_test = y_list[0], y_list[1]
        
        print(f"\n{subject_id}: Train trials {len(X_train)}, Test trials {len(X_test)}")
        print(f"Class distribution - Train: {np.bincount(y_train)}, Test: {np.bincount(y_test)}")
        
        return (X_train, y_train), (X_test, y_test)
    else:
        print(f"  Warning: Only {len(X_list)} session(s) loaded successfully")
        # Return what we have
        if len(X_list) == 1:
            return (X_list[0], y_list[0]), (X_list[0], y_list[0])
        else:
            return None

In [165]:
subjects = [f'A0{i}' for i in range(1, 10)]   # A01 to A09
results = {}

for subj in subjects:
    print(f"\n=== Processing subject {subj} ===")

    try:
        data = load_subject_data(subj, DATA_DIR)
        if data is None:
            print(f"Skipping subject {subj} - no valid data")
            continue
            
        (X_train_full, y_train_full), (X_test, y_test) = data
        
        # Debug: Check data
        print(f"  Training data shape: {X_train_full.shape}")
        print(f"  Test data shape: {X_test.shape}")
        print(f"  Training class distribution: {np.bincount(y_train_full)}")
        print(f"  Test class distribution: {np.bincount(y_test)}")
        
        # Check if we have enough data
        if len(X_train_full) < 20:
            print(f"  Warning: Only {len(X_train_full)} training samples")
        
        # Handle imbalanced classes
        class_counts = np.bincount(y_train_full)
        print(f"  Class counts: {class_counts}")
        
        # Check if any class has less than 2 samples
        if np.any(class_counts < 2):
            print(f"  Some classes have < 2 samples, using non-stratified split")
            X_train, X_val, y_train, y_val = train_test_split(
                X_train_full, y_train_full, test_size=0.2, random_state=42)
        else:
            # Use stratified split
            X_train, X_val, y_train, y_val = train_test_split(
                X_train_full, y_train_full, test_size=0.2, 
                stratify=y_train_full, random_state=42)
        
        print(f"  Train split: {len(X_train)} samples")
        print(f"  Val split: {len(X_val)} samples")
        print(f"  Train classes after split: {np.bincount(y_train)}")
        
        # Check if any class disappeared in split
        if len(np.unique(y_train)) < len(np.unique(y_train_full)):
            print(f"  Warning: Some classes missing in training split")
            # Consider oversampling or adjust model
        
    except Exception as e:
        print(f"Error processing {subj}: {e}")
        import traceback
        traceback.print_exc()
        continue

    train_ds = EEGDataset(X_train, y_train)
    val_ds   = EEGDataset(X_val,   y_val)
    test_ds  = EEGDataset(X_test,  y_test)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE)
    test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE)

    model = BiGRUModel().to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LR)

    best_val_acc = 0.0
    for epoch in range(1, EPOCHS + 1):
        model.train()
        for data, target in train_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        val_preds, val_true = [], []
        with torch.no_grad():
            for data, target in val_loader:
                data = data.to(DEVICE)
                output = model(data)
                pred = output.argmax(dim=1)
                val_preds.extend(pred.cpu().numpy())
                val_true.extend(target.numpy())
        val_acc = accuracy_score(val_true, val_preds)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f'best_model_{subj}.pth')

        if epoch % 10 == 0 or epoch == EPOCHS:
            print(f"Epoch {epoch:2d} | Val Acc: {val_acc:.4f}")

    # Load best model and test on E session
    model.load_state_dict(torch.load(f'best_model_{subj}.pth'))
    model.eval()
    test_preds, test_true = [], []
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(DEVICE)
            output = model(data)
            pred = output.argmax(dim=1)
            test_preds.extend(pred.cpu().numpy())
            test_true.extend(target.numpy())

    acc  = accuracy_score(test_true, test_preds)
    kappa = cohen_kappa_score(test_true, test_preds)

    results[subj] = {'accuracy': acc, 'kappa': kappa}
    print(f"{subj} Test Accuracy: {acc:.4f} | Kappa: {kappa:.4f}")

# ==================== SUMMARY ====================
accs  = [v['accuracy'] for v in results.values()]
kappas = [v['kappa'] for v in results.values()]

print("\n=== FINAL RESULTS (Subject-Dependent) ===")
for subj, res in results.items():
    print(f"{subj}: Acc = {res['accuracy']:.4f}, Kappa = {res['kappa']:.4f}")

print(f"\nAverage Accuracy: {np.mean(accs):.4f} (±{np.std(accs):.3f})")
print(f"Average Kappa:    {np.mean(kappas):.4f} (±{np.std(kappas):.3f})")


=== Processing subject A01 ===
Extracting GDF parameters from BCICIV_2a/A01T.gdf...
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


c:\Users\MrKillShOtzz\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: [np.str_('1023'), np.str_('1072'), np.str_('276'), np.str_('277'), np.str_('32766'), np.str_('768'), np.str_('769'), np.str_('770'), np.str_('771'), np.str_('772')]

Session T:
  Total events: 603
  Unique event codes: [ 1  2  3  4  5  6  7  8  9 10]
  Event IDs from annotations: {np.str_('1023'):

c:\Users\MrKillShOtzz\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: [np.str_('1023'), np.str_('1072'), np.str_('276'), np.str_('277'), np.str_('32766'), np.str_('768'), np.str_('783')]

Session E:
  Total events: 595
  Unique event codes: [1 2 3 4 5 6 7]
  Event IDs from annotations: {np.str_('1023'): 1, np.str_('1072'): 2, np.str_('276'): 3, np.str_('277'): 4, np

...

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: [np.str_('1023'), np.str_('1072'), np.str_('276'), np.str_('277'), np.str_('32766'), np.str_('768'), np.str_('783')]

Session E:
  Total events: 605
  Unique event codes: [1 2 3 4 5 6 7]
  Event IDs from annotations: {np.str_('1023'): 1, np.str_('1072'): 2, np.str_('276'): 3, np.str_('277'): 4, np

c:\Users\MrKillShOtzz\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: [np.str_('1023'), np.str_('1072'), np.str_('276'), np.str_('277'), np.str_('32766'), np.str_('768'), np.str_('769'), np.str_('770'), np.str_('771'), np.str_('772')]

Session T:
  Total events: 639
  Unique event codes: [ 1  2  3  4  5  6  7  8  9 10]
  Event IDs from annotations: {np.str_('1023'):

c:\Users\MrKillShOtzz\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: [np.str_('1023'), np.str_('1072'), np.str_('276'), np.str_('277'), np.str_('32766'), np.str_('768'), np.str_('783')]

Session E:
  Total events: 612
  Unique event codes: [1 2 3 4 5 6 7]
  Event IDs from annotations: {np.str_('1023'): 1, np.str_('1072'): 2, np.str_('276'): 3, np.str_('277'): 4, np